In [1]:
from pyspark.sql import SparkSession
from vars import *
from datetime import date
from functions import loadConfigs
import os
from pyspark.sql.functions import explode
from pyspark.sql.functions import lit
from pyspark.sql import functions as F
from delta import *

# minio credentials
minio_endpoint = "http://minio:9000"
minio_access_key = "iQekrU3nA33aKktu2GMo"
minio_secret_key = "1OGQ3m1lWEZVQ31xKfPCKmZVFVb2lXwCorjZr1d1"
minio_region = "us-east-1"
minio_bucket = "reddit-datalake"

builder = loadConfigs(SparkSession.builder)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
today = date.today().strftime('%Y%m%d')

df = spark.read.format("delta").load(f"s3a://{minio_bucket}/silver/gildings/author_gildings")

In [3]:
df.show()

+--------------------+------------+--------------------+-----------------+---------------+-------------------+-----------+--------------+----------+
|              author|subreddit_id|           subreddit|total_gild_silver|total_gild_gold|total_gild_platinum|total_score|total_gildings|updated_at|
+--------------------+------------+--------------------+-----------------+---------------+-------------------+-----------+--------------+----------+
|     SpicyThunder335|   t5_4yrnne|            ModCoord|                2|              5|                  5|      13730|            12|  20230614|
|           timmeh519|    t5_2qh5i|               happy|                1|              5|                  2|     125457|             8|  20230614|
|          Riftmarked|    t5_3g1qk|       goldenknights|                0|              1|                  2|       1746|             3|  20230614|
|          ICumCoffee|    t5_2qh3l|                news|                0|              1|                

In [11]:
from pyspark.sql.functions import col, sum

df_gold = df.withColumn("total_gild_count", col("total_gildings")) \
                   .filter(col("total_gild_count") > 0) \
                   .select("author", "subreddit_id", "subreddit", "total_gild_count", "total_score", "updated_at")

In [12]:
df_gold.show()

+--------------------+------------+--------------------+----------------+-----------+----------+
|              author|subreddit_id|           subreddit|total_gild_count|total_score|updated_at|
+--------------------+------------+--------------------+----------------+-----------+----------+
|     SpicyThunder335|   t5_4yrnne|            ModCoord|              12|      13730|  20230614|
|           timmeh519|    t5_2qh5i|               happy|               8|     125457|  20230614|
|          Riftmarked|    t5_3g1qk|       goldenknights|               3|       1746|  20230614|
|          ICumCoffee|    t5_2qh3l|                news|               2|      46413|  20230614|
|                PLPQ|    t5_2s8x6|            castiron|               1|      11829|  20230614|
|    Eastern-Truck433|    t5_2qh5i|               happy|               1|      12734|  20230614|
|       happysheeple3|   t5_43zwn1|          Superstonk|               1|       2995|  20230614|
|         Goldeneye07|    t5_3